<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/Blender_Renderer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt_text](https://raw.githubusercontent.com/qsardor/GoogleColabProjects/refs/heads/main/googlecolabimage.png)

# 🎬 **Render Blender files with Google Colab (Pro Edition)**
> **Last updated:** 12.2025

* **Blender 5.0 & LTS Versions** are now supported! 📚
* **Robust GPU Detection:** Automatically handles Tesla K80/T4/A100 optimization. 👷
* **Streaming Logs:** See your render progress in real-time. ⏱️

## **📝 Notes:**
* Run the cells one by one, from top to bottom. ▶️
* To copy a file's path, use the file explorer on the left, click the three dots (⋮) next to the file, and select `Copy path`. 🗄️
* **Spaces in filenames are now supported!** You don't need to rename your files. 📂
* If you upload your `.blend` file directly, please wait for the upload to complete before running the cells. 🔽
* **⚠️ Free Tier Warning:** Google Colab's free sessions have strict time limits. For long animation sequences, use the `gdrive_output` option to save your renders directly to Google Drive.

In [ ]:
#@title ☁️ 1. Mount Google Drive (Optional)
#@markdown Run this cell if your files are in Google Drive or you want to save your renders there.
from google.colab import drive

try:
  drive.mount("/content/drive", force_remount=True)
  print("✅ Google Drive mounted successfully!")
except Exception as e:
  print(f"⚠️ Could not mount Google Drive: {e}")

In [ ]:
#@title ⚙️ 2. Configure Your Render Job

#@markdown ###  Blender Options
#@markdown Select the version. Defaults to the latest stable LTS.
blender_version = '5.0.0' #@param ['2.83.20', '2.93.18', '3.0.1', '3.1.2', '3.2.2', '3.3.20', '3.4.1', '3.5.1', '3.6.23', '4.0.2', '4.1.1', '4.2.15', '4.3.2', '4.4.1', '4.5.5', '5.0.0'] {allow-input: true}
project_name = 'MyBlenderProject' #@param {type: 'string'}
blend_file_path = '/content/File.blend' #@param {type: 'string'}

#@markdown ### 🧩 Add-on Options
#@markdown Check this box to install all .zip files from the `add-ons` folder.
install_addons = False #@param {type:"boolean"}

#@markdown ### ☁️ Google Drive Options
gdrive_output = False #@param {type:"boolean"}
organize_gdrive = False #@param {type:"boolean"}

#@markdown ### 💻 Hardware Options
gpu_enabled = True #@param {type:"boolean"}
optix_enabled = True #@param {type:"boolean"}

#@markdown ---
#@markdown ### ⚙️ Render Settings Source
#@markdown Choose whether to use the settings saved in your `.blend` file or the manual overrides below.
render_settings_source = 'Use Settings from .blend File' #@param ["Use Settings from .blend File", "Use Manual Overrides"]

#@markdown ### ✨ Manual Overrides (only apply if 'Use Manual Overrides' is selected)
#@markdown > These settings are **ignored** unless you change the option above.
render_engine = 'CYCLES' #@param ["CYCLES", "EEVEE"]
use_denoising = True #@param {type:"boolean"}
render_animation_sequence = True #@param {type:"boolean"}
#@markdown &nbsp;
width = 1920 #@param {type: 'integer'}
height = 1080 #@param {type: 'integer'}
samples = 256 #@param {type: 'integer'}
start_frame = 1 #@param {type: 'integer'} # If rendering a single frame, this is the frame that will be rendered.
end_frame = 100 #@param {type: 'integer'}
output_format = 'PNG' #@param ['PNG', 'OPEN_EXR', 'JPEG', 'TIFF', 'FFMPEG (Video Render)']

In [ ]:
#@title 🔍 3. Review Settings & Prepare to Render
#@markdown This cell prepares everything and shows you a summary of your settings. **It does not start the render.**

import os
import sys
import shutil
import subprocess
from pathlib import Path

# --- Global Variable for Command (Now a list for safety) ---
render_cmd = []

# --- Helper Functions ---
def create_project_dirs():
    os.makedirs(f"/content/{project_name}/rendered", exist_ok=True)
    os.makedirs(f"/content/{project_name}/blend", exist_ok=True)
    if Path('/content/sample_data').exists():
        shutil.rmtree('/content/sample_data')

def setup_gdrive_dirs():
    if gdrive_output:
        os.makedirs(f"/content/drive/MyDrive/BlenderColab/{project_name}/rendered", exist_ok=True)
    if organize_gdrive and "/content/drive/MyDrive" in blend_file_path:
        os.makedirs(f"/content/drive/MyDrive/BlenderColab/{project_name}/blend", exist_ok=True)
        shutil.move(blend_file_path, f"/content/drive/MyDrive/BlenderColab/{project_name}/blend/")
        print("Moved .blend file to organized GDrive folder.")

def check_and_copy_blend_file():
    print("🔍 Checking for .blend file...")
    blend_path = Path(blend_file_path)
    if not blend_path.exists():
        sys.exit(f"❌ ERROR: Blend file not found at: {blend_file_path}")
    try:
        with open(blend_path, 'rb') as f:
            header = f.read(12)
            if not header.startswith(b'BLENDER'):
                sys.exit(f"❌ ERROR: The file at {blend_file_path} is not a valid .blend file.")
            version_str = header[9:12].decode('ascii')
            blender_file_version = f"{version_str[0]}.{version_str[1:]}"
            print(f"✅ Blend file found and valid. Saved with Blender version: {blender_file_version}")
    except Exception as e:
        sys.exit(f"❌ ERROR: Could not read the .blend file. Details: {e}")

    dest_path = f"/content/{project_name}/blend/{os.path.basename(blend_file_path)}"
    print(f"   ↳ Copying to project folder...")
    shutil.copy(blend_file_path, dest_path)

def get_gpu_info():
    global optix_enabled, gpu_enabled
    print("\n💻 Checking GPU info...")

    # 1. Check if NVIDIA-SMI exists
    if not shutil.which('nvidia-smi'):
        print("❌ No GPU detected! (nvidia-smi not found)")
        print("   ⚠️ Switching to CPU rendering. This will be very slow.")
        gpu_enabled = False
        optix_enabled = False
        return

    try:
        # 2. Get the Name (Display only)
        gpu_name = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], encoding='utf-8').strip()
        print(f"✅ Current GPU: {gpu_name}")

        # 3. Get Compute Capability (Robust Check)
        compute_cap = subprocess.check_output(['nvidia-smi', '--query-gpu=compute_cap', '--format=csv,noheader'], encoding='utf-8').strip()
        cap_ver = float(compute_cap)
        print(f"   ↳ Compute Capability: {cap_ver}")

        # 4. Logical Check for OptiX (Maxwell+ requires 5.0+)
        if cap_ver < 5.0 and optix_enabled:
            print(f"⚠️ GPU architecture is too old for OptiX (Cap {cap_ver} < 5.0).")
            print("   ↳ Disabling OptiX to prevent crashes.")
            optix_enabled = False
    except Exception as e:
        print(f"⚠️ Error reading GPU info: {e}")
        print("   Defaulting to safe settings.")
        optix_enabled = False

def download_blender():
    blender_path = Path(f"/content/{blender_version}")
    if blender_path.exists():
        print(f"\n📚 Blender {blender_version} already downloaded.")
        return

    # 💎 UPDATED BLENDER REPOSITORY (As of Dec 2025)
    blender_url_dict = {
        '5.0.0': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender5.0/blender-5.0.0-linux-x64.tar.xz",
        '4.5.5': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender4.5/blender-4.5.5-linux-x64.tar.xz",
        '4.4.1': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender4.4/blender-4.4.1-linux-x64.tar.xz",
        '4.3.2': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender4.3/blender-4.3.2-linux-x64.tar.xz",
        '4.2.15': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender4.2/blender-4.2.15-linux-x64.tar.xz",
        '4.1.1': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender4.1/blender-4.1.1-linux-x64.tar.xz",
        '4.0.2': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender4.0/blender-4.0.2-linux-x64.tar.xz",
        '3.6.23': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.6/blender-3.6.23-linux-x64.tar.xz",
        '3.5.1': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.5/blender-3.5.1-linux-x64.tar.xz",
        '3.4.1': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.4/blender-3.4.1-linux-x64.tar.xz",
        '3.3.20': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.3/blender-3.3.20-linux-x64.tar.xz",
        '3.2.2': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.2/blender-3.2.2-linux-x64.tar.xz",
        '3.1.2': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.1/blender-3.1.2-linux-x64.tar.xz",
        '3.0.1': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.0/blender-3.0.1-linux-x64.tar.xz",
        '2.93.18': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.93/blender-2.93.18-linux-x64.tar.xz",
        '2.83.20': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.83/blender-2.83.20-linux-x64.tar.xz",
    }

    print(f"\n🔽 Downloading Blender {blender_version}...")
    blender_url = blender_url_dict.get(blender_version, f"https://ftp.nluug.nl/pub/graphics/blender/release/Blender{blender_version.rpartition('.')[0]}/blender-{blender_version}-linux-x64.tar.xz")
    archive_name = os.path.basename(blender_url)

    subprocess.run(["wget", "-q", "--show-progress", blender_url], check=True)

    print("   ↳ Extracting...")
    os.makedirs(blender_path, exist_ok=True)
    subprocess.run(["tar", "-xkf", archive_name, "-C", str(blender_path), "--strip-components=1"], check=True)
    os.remove(archive_name)
    print("✅ Blender setup complete.")

def create_gpu_script():
    script_content = f"""import re\nimport bpy\nscene = bpy.context.scene\nscene.cycles.device = 'GPU'\nprefs = bpy.context.preferences\nprefs.addons['cycles'].preferences.get_devices()\ncprefs = prefs.addons['cycles'].preferences\nfor compute_device_type in ('OPTIX', 'CUDA', 'HIP', 'METAL', 'ONEAPI', 'OPENCL', 'NONE'):\n    try:\n        cprefs.compute_device_type = compute_device_type\n        break\n    except TypeError:\n        pass\nfor device in cprefs.devices:\n    if not re.match('intel', device.name, re.I):\n        device.use = {gpu_enabled}\n    else:\n        device.use = False\n"""
    with open('setgpu.py', 'w') as f:
        f.write(script_content)

def build_and_summarize_command():
    global render_cmd
    blend_file_name = os.path.basename(blend_file_path)
    local_blend_path = f'/content/{project_name}/blend/{blend_file_name}'
    output_dir = f'/content/drive/MyDrive/BlenderColab/{project_name}/rendered/' if gdrive_output else f'/content/{project_name}/rendered/'
    output_path = f'{output_dir}{project_name}_####'

    final_render_engine = render_engine if render_settings_source == 'Use Manual Overrides' else 'CYCLES'

    # Start building the command list (Handles spaces in paths safely)
    cmd = ['./blender', '-b', local_blend_path, '-noaudio', '-E', final_render_engine]

    if final_render_engine == 'CYCLES':
        device_type = "OPTIX" if optix_enabled else "CUDA"
        cmd.extend(['-P', 'setgpu.py', '--', '--cycles-device', device_type])

    cmd.extend(['-o', output_path])

    py_overrides = []
    summary = []

    # --- Add-on Installation (Robust Unzip Method) ---
    if install_addons:
        gdrive_addon_dir = f'/content/drive/MyDrive/BlenderColab/add-ons'
        local_addon_dir = '/content/add-ons'
        addon_dir_to_use = None

        if os.path.exists(gdrive_addon_dir) and os.listdir(gdrive_addon_dir):
            addon_dir_to_use = gdrive_addon_dir
        elif os.path.exists(local_addon_dir) and os.listdir(local_addon_dir):
             addon_dir_to_use = local_addon_dir

        if addon_dir_to_use:
            zip_files = [f for f in os.listdir(addon_dir_to_use) if f.endswith('.zip')]
            if zip_files:
                summary.append(f"Installing {len(zip_files)} add-on(s) from {addon_dir_to_use}")

                # Calculate path: /content/5.0.0/5.0/scripts/addons
                # Note: blender_version might be 5.0.0 but folder is 5.0
                major_minor = ".".join(blender_version.split(".")[:2])
                addons_dest = f"/content/{blender_version}/{major_minor}/scripts/addons"
                os.makedirs(addons_dest, exist_ok=True)

                py_overrides.append('import bpy')
                for addon_zip in zip_files:
                    zip_path = os.path.join(addon_dir_to_use, addon_zip)
                    # 1. Direct Unzip (Solves naming issues)
                    subprocess.run(['unzip', '-o', '-q', zip_path, '-d', addons_dest], check=False)

                    # 2. Try to enable (Assuming zip name ~ module name)
                    addon_module_name = addon_zip.replace('.zip', '')
                    py_overrides.append(f'bpy.ops.preferences.addon_enable(module=\"{addon_module_name}\")')

    # --- Manual Settings ---
    if render_settings_source == 'Use Manual Overrides':
        summary.append("Using Manual Overrides")
        summary.append(f"Render Engine: {render_engine}")
        summary.append(f"Resolution: {width}x{height}")
        summary.append(f"Samples: {samples}")

        if render_engine == 'CYCLES':
          summary.append(f"Denoising: {'On' if use_denoising else 'Off'}")
          py_overrides.append(f'bpy.context.scene.cycles.use_denoising = {use_denoising}')

        if render_animation_sequence:
            cmd.extend(['-s', str(start_frame), '-e', str(end_frame), '-a'])
            summary.append(f"Mode: Animation | Frames: {start_frame} to {end_frame}")
        else:
            cmd.extend(['-f', str(start_frame)])
            summary.append(f"Mode: Single Frame | Frame: {start_frame}")

        api_output_format = 'FFMPEG' if 'FFMPEG' in output_format else output_format
        py_overrides.append(f'bpy.context.scene.render.image_settings.file_format = \"{api_output_format}\" ')

        if width > 0 and height > 0:
            py_overrides.append(f'bpy.context.scene.render.resolution_x = {width}')
            py_overrides.append(f'bpy.context.scene.render.resolution_y = {height}')

        if samples > 0 and render_engine == 'CYCLES':
            py_overrides.append(f'bpy.context.scene.cycles.samples = {samples}')
        elif samples > 0 and render_engine == 'EEVEE':
            py_overrides.append(f'bpy.context.scene.eevee.taa_render_samples = {samples}')

    else:
        cmd.append('-a')
        summary.append("Using settings from .blend file")

    if py_overrides:
        cmd.extend(['--python-expr', '; '.join(py_overrides)])

    render_cmd = cmd # Store list globally

    print("\n" + "-"*50)
    print("          Render Job Summary")
    print("-"*50)
    print(f"  Project Name: {project_name}")
    print(f"  Blender Version: {blender_version}")
    print(f"  Output Location: {output_dir}")
    for item in summary:
        print(f"  - {item}")
    print("-"*50)
    print("\n⚠️  Please review the settings above carefully.")
    print("   If everything is correct, run the next cell to start the render.")
    print("-"*50)

# --- Main Execution ---
create_project_dirs()
if gdrive_output or organize_gdrive:
    setup_gdrive_dirs()
check_and_copy_blend_file()
get_gpu_info()
download_blender()
create_gpu_script()
build_and_summarize_command()


In [ ]:
#@title 🚀 4. START THE RENDER!
#@markdown Run this cell **after** reviewing the summary above to start the render process.
import datetime
import os
import subprocess
import shutil

timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_filename = f"render_log_{timestamp}.txt"
output_location = f'/content/drive/MyDrive/BlenderColab/{project_name}/rendered/' if gdrive_output else f'/content/{project_name}/rendered/'
log_filepath = os.path.join(output_location, log_filename)

# --- 1. Disk Space Safety Check ---
total, used, free = shutil.disk_usage("/")
free_gb = free // (2**30)
print(f"💾 Free Disk Space: {free_gb} GB")
if free_gb < 2:
    print("⚠️ WARNING: You have less than 2GB of space left!")
    print("   If you are rendering a long animation, your session might crash due to full disk.")
    print("   Consider connecting Google Drive and enabling 'gdrive_output'.")

# --- 2. Start Render (Streaming) ---
print("\n" + "-"*80)
print("🚀 Starting Blender Render (Streaming Output)...")
print(f"   Full logs are also saving to: {log_filename}")
print("-"*80 + "\n")

os.chdir(f'/content/{blender_version}')

# Open the log file
with open(log_filepath, "w") as log_file:
    # Start the process with direct piping
    process = subprocess.Popen(
        render_cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT, # Merge stderr into stdout
        text=True,
        bufsize=1 # Line buffering
    )

    # Stream output line by line
    for line in process.stdout:
        print(line, end="") # Print to Colab Console
        log_file.write(line) # Write to Log File

    # Wait for completion
    process.wait()

print("\n" + "-"*80)
if process.returncode == 0:
    print("✅ Render Finished Successfully!")
    print(f"   Check your output files in: {output_location}")
else:
    print(f"❌ Render Failed with return code {process.returncode}")
    print(f"   Check the log file: {log_filename}")
print("-"*80 + "\n")

In [ ]:
#@title 📦 5. Zip and Download Renders (Optional)
#@markdown Run this if you didn't use Google Drive and want to download your renders and logs.

import os
from google.colab import files

local_render_path = f'/content/{project_name}/rendered'
zip_path = f'/content/{project_name}_render_output.zip'

if os.path.exists(local_render_path) and any(os.scandir(local_render_path)):
    print("Zipping files...")
    os.system(f'zip -r -j "{zip_path}" "{local_render_path}"')
    print(f'✅ Created zip file at: {zip_path}')
    files.download(zip_path)
else:
    print("🤔 Render folder not found or is empty. Did the render complete successfully?")


### ❤️ **Credits**
* **Author:** QSARDOR ([✈️ Telegram](https://t.me/qsardorblog) | [💻 GitHub](https://github.com/qsardor/))
* **More Cool Stuff:** [Google Colab Projects Repo](https://github.com/qsardor/GoogleColabProjects)